# Implementação com Threads OpenMP

## Código-fonte modificado

In [ ]:
# O OpenMP simplifica a criação de threads e a coordenação entre elas.

## Justificativa da alteração


## Speed up

Speed up obtido

Gráficos e tabelas para mostrar que sua implementação em openMP valeu a pena.

Observe que mesmo assim você não vai conseguir resolver exaustivamente o problema, mas o tamanho do grafo será maior, e o tempo de processamento para um mesmo tamanho deve ser preferencialmente menor.